In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving combined_matrix.csv to combined_matrix.csv


In [ ]:
import pandas as pd
import numpy as np
import keras
from keras import layers
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [ ]:
sorted_combined_matrix = pd.read_csv("combined_matrix.csv")

In [ ]:
sorted_combined_matrix

,parameter_location_id,2019-10-28T02:00:00+08:00,2018-05-28T12:00:00+08:00,2018-02-23T17:00:00+08:00,2018-03-25T14:00:00+08:00,2019-11-14T11:00:00+08:00,2021-06-01T17:00:00+08:00,2018-09-09T12:00:00+08:00,2018-12-04T01:00:00+08:00,2019-05-22T20:00:00+08:00,2020-05-07T23:00:00+08:00,2018-01-11T05:00:00+08:00,2019-08-21T12:00:00+08:00,2019-01-10T09:00:00+08:00,2019-01-15T15:00:00+08:00,2020-12-09T09:00:00+08:00,2020-04-08T13:00:00+08:00,2018-11-16T17:00:00+08:00,2020-02-17T06:00:00+08:00,2018-01-15T21:00:00+08:00,2018-12-30T04:00:00+08:00,2019-11-12T09:00:00+08:00,2021-06-11T05:00:00+08:00,2019-11-09T00:00:00+08:00,2020-08-05T04:00:00+08:00,2019-12-11T15:00:00+08:00,2019-08-29T19:00:00+08:00,2018-03-21T13:00:00+08:00,2019-11-26T18:00:00+08:00,2018-06-30T16:00:00+08:00,2021-02-02T17:00:00+08:00,2019-08-03T07:00:00+08:00,2019-04-25T20:00:00+08:00,2019-04-03T12:00:00+08:00,2019-12-29T18:00:00+08:00,2021-01-17T23:00:00+08:00,2020-05-28T00:00:00+08:00,2019-02-12T05:00:00+08:00,2021-04-22T14:00:00+08:00,2019-12-12T13:00:00+08:00,2020-10-26T07:00:00+08:00,2019-08-21T05:00:00+08:00,2019-06-07T21:00:00+08:00,2019-08-17T14:00:00+08:00,2018-05-21T21:00:00+08:00,2019-03-15T15:00:00+08:00,2021-05-07T05:00:00+08:00,2018-02-13T12:00:00+08:00,2020-04-03T06:00:00+08:00,2018-01-23T22:00:00+08:00,...,2018-05-29T19:00:00+08:00,2019-03-19T03:00:00+08:00,2020-07-27T13:00:00+08:00,2018-08-01T09:00:00+08:00,2019-10-13T19:00:00+08:00,2020-10-10T14:00:00+08:00,2021-07-05T04:00:00+08:00,2021-05-11T23:00:00+08:00,2021-05-15T12:00:00+08:00,2019-04-17T04:00:00+08:00,2018-12-01T19:00:00+08:00,2021-05-15T06:00:00+08:00,2020-11-02T00:00:00+08:00,2019-01-14T00:00:00+08:00,2020-02-05T13:00:00+08:00,2021-07-13T18:00:00+08:00,2020-07-10T16:00:00+08:00,2020-03-19T10:00:00+08:00,2020-01-26T20:00:00+08:00,2020-11-11T22:00:00+08:00,2020-04-06T13:00:00+08:00,2020-02-21T13:00:00+08:00,2020-06-22T04:00:00+08:00,2018-11-26T01:00:00+08:00,2020-09-04T00:00:00+08:00,2019-05-20T10:00:00+08:00,2019-06-21T22:00:00+08:00,2019-07-05T17:00:00+08:00,2021-03-06T01:00:00+08:00,2021-04-24T02:00:00+08:00,2020-10-16T11:00:00+08:00,2019-11-13T17:00:00+08:00,2021-06-15T23:00:00+08:00,2020-02-17T16:00:00+08:00,2020-05-26T09:00:00+08:00,2019-03-24T19:00:00+08:00,2021-01-25T17:00:00+08:00,2021-02-05T11:00:00+08:00,2019-11-26T06:00:00+08:00,2018-08-09T00:00:00+08:00,2019-08-13T15:00:00+08:00,2020-12-07T18:00:00+08:00,2021-01-15T20:00:00+08:00,2021-03-24T13:00:00+08:00,2018-08-17T02:00:00+08:00,2020-02-08T22:00:00+08:00,2019-04-18T07:00:00+08:00,2020-12-28T06:00:00+08:00,2020-04-14T11:00:00+08:00,2021-01-09T12:00:00+08:00
0,co_6168,1400.0,769.0,680.0,796.0,400.0,700.0,363.0,2000.0,800.0,600.0,1200.0,600.0,600.0,2300.0,1300.0,300.0,1000.0,300.0,3070.0,500.0,800.0,700.0,900.0,400.0,700.0,700.0,559.0,600.0,603.0,1100.0,500.0,700.0,700.0,600.0,800.0,500.0,800.0,700.0,700.0,800.0,800.0,600.0,600.0,1024.0,300.0,500.0,675.0,400.0,650.0,...,846.0,600.0,800.0,491.0,800.0,400.0,700.0,400.0,400.0,900.0,1500.0,500.0,900.0,1300.0,600.0,500.0,500.0,300.0,800.0,1000.0,600.0,500.0,500.0,1800.0,800.0,300.0,800.0,500.0,1100.0,800.0,400.0,400.0,500.0,200.0,800.0,700.0,900.0,900.0,300.0,536.0,400.0,800.0,1100.0,500.0,752.0,300.0,800.0,1100.0,500.0,500.0
1,no2_6168,73.0,65.0,44.0,49.0,55.0,22.0,14.0,45.0,97.0,38.0,98.0,14.0,42.0,78.0,60.0,42.0,49.0,24.0,142.0,33.0,82.0,58.0,116.0,27.0,51.0,42.0,30.0,55.0,55.0,52.0,33.0,31.0,44.0,69.0,86.0,41.0,29.0,22.0,50.0,84.0,57.0,40.0,18.0,53.0,23.0,57.0,45.0,41.0,40.0,...,78.0,46.0,35.0,29.0,87.0,16.0,26.0,17.0,19.0,98.0,82.0,28.0,92.0,76.0,16.0,21.0,0.0,25.0,33.0,108.0,36.0,22.0,13.0,138.0,115.0,26.0,63.0,26.0,69.0,17.0,26.0,35.0,37.0,8.0,42.0,74.0,35.0,53.0,27.0,48.0,22.0,63.0,89.0,36.0,12.0,17.0,80.0,68.0,46.0,29.0
2,o3_6168,49.0,78.0,76.0,113.0,63.0,243.0,130.0,21.0,199.0,119.0,6.0,203.0,25.0,33.0,14.0,81.0,54.0,80.0,9.0,47.0,111.0,124.0,123.0,129.0,82.0,164.0,99.0,23.0,52.0,63.0,223.0,118.0,128.0,48.0,52.0,195.0,55.0,92.0,82.0,118.0,203.0,175.0,202.0,19.0,132.0,116.0,80.

In [ ]:
sorted_combined_matrix = sorted_combined_matrix.set_index('parameter_location_id')

sorted_combined_matrix_T =sorted_combined_matrix.T
sorted_combined_matrix_T.replace(to_replace=0, method='ffill', inplace=True)
sorted_combined_matrix_T.index

Index(['2019-10-28T02:00:00+08:00', '2018-05-28T12:00:00+08:00',
       '2018-02-23T17:00:00+08:00', '2018-03-25T14:00:00+08:00',
       '2019-11-14T11:00:00+08:00', '2021-06-01T17:00:00+08:00',
       '2018-09-09T12:00:00+08:00', '2018-12-04T01:00:00+08:00',
       '2019-05-22T20:00:00+08:00', '2020-05-07T23:00:00+08:00',
       ...
       '2019-08-13T15:00:00+08:00', '2020-12-07T18:00:00+08:00',
       '2021-01-15T20:00:00+08:00', '2021-03-24T13:00:00+08:00',
       '2018-08-17T02:00:00+08:00', '2020-02-08T22:00:00+08:00',
       '2019-04-18T07:00:00+08:00', '2020-12-28T06:00:00+08:00',
       '2020-04-14T11:00:00+08:00', '2021-01-09T12:00:00+08:00'],
      dtype='object', length=25941)

In [ ]:
# Normalize the combined_matrix data
normalized_data = sorted_combined_matrix_T
scaler = StandardScaler()
scaled_dataset = scaler.fit_transform(normalized_data)

In [ ]:
optimizers = ['adam', 'rmsprop']
activation_functions = ['relu', 'sigmoid', 'tanh', 'elu', 'selu', 'softmax', 'swish']
mse_values = []
window_size = 3

In [ ]:
# Define a function to create supervised data
def to_supervised(train):
    X = []
    Y = []
    for i in range(window_size, len(train)):
        X.append(train[i - window_size:i, :])
        Y.append(train[i, 0:1])
    return X, Y

In [ ]:

for optimizer in optimizers:
    for activation in activation_functions:
        # Load or create your 'scaled_dataset' here

        # Create supervised data
        X, Y = to_supervised(scaled_dataset)
        X = np.array(X)
        Y = np.array(Y)

        # Split the data into training and testing sets
        test_size = 0.2
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=42)

        # Build the LSTM model
        model = Sequential()
        model.add(GRU(units=100, return_sequences=True, input_shape=(window_size, X_train.shape[2]), activation=activation))
        model.add(GRU(units=100, return_sequences=True, activation=activation))
        model.add(GRU(units=100, activation=activation))
        model.add(Dense(units=100))
        model.add(Dense(units=X_train.shape[2]))
        model.compile(optimizer=optimizer, loss='mean_squared_error')

        # Train the model
        lstm_history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100, batch_size=256, verbose=2)

        # Make predictions and calculate MSE
        y_pred = model.predict(X_test)
        y_pred_reduced = y_pred.mean(axis=1)
        mse = mean_squared_error(Y_test, y_pred_reduced)

        print("Average MSE for the " + str(optimizer) + " optimizer and " + str(activation) + " activation function:", mse)
        mse_values.append(mse)

Epoch 1/100
82/82 - 11s - loss: 0.9762 - val_loss: 1.0964 - 11s/epoch - 134ms/step
Epoch 2/100
82/82 - 2s - loss: 0.9763 - val_loss: 1.0955 - 2s/epoch - 21ms/step
Epoch 3/100
82/82 - 2s - loss: 0.9754 - val_loss: 1.0953 - 2s/epoch - 21ms/step
Epoch 4/100
82/82 - 2s - loss: 0.9743 - val_loss: 1.1007 - 2s/epoch - 21ms/step
Epoch 5/100
82/82 - 2s - loss: 0.9712 - val_loss: 1.0966 - 2s/epoch - 21ms/step
Epoch 6/100
82/82 - 2s - loss: 0.9666 - val_loss: 1.0982 - 2s/epoch - 20ms/step
Epoch 7/100
82/82 - 2s - loss: 0.9594 - val_loss: 1.1053 - 2s/epoch - 20ms/step
Epoch 8/100
82/82 - 2s - loss: 0.9499 - val_loss: 1.1107 - 2s/epoch - 21ms/step
Epoch 9/100
82/82 - 2s - loss: 0.9446 - val_loss: 1.1168 - 2s/epoch - 22ms/step
Epoch 10/100
82/82 - 2s - loss: 0.9208 - val_loss: 1.1186 - 2s/epoch - 19ms/step
Epoch 11/100
82/82 - 2s - loss: 0.8981 - val_loss: 1.1563 - 2s/epoch - 20ms/step
Epoch 12/100
82/82 - 2s - loss: 0.8715 - val_loss: 1.1814 - 2s/epoch - 20ms/step
Epoch 13/100
82/82 - 2s - loss: 0.

Epoch 1/100
82/82 - 6s - loss: 0.9872 - val_loss: 1.0983 - 6s/epoch - 78ms/step
Epoch 2/100
82/82 - 2s - loss: 0.9773 - val_loss: 1.0958 - 2s/epoch - 20ms/step
Epoch 3/100
82/82 - 2s - loss: 0.9767 - val_loss: 1.1034 - 2s/epoch - 20ms/step
Epoch 4/100
82/82 - 2s - loss: 0.9770 - val_loss: 1.0957 - 2s/epoch - 20ms/step
Epoch 5/100
82/82 - 2s - loss: 0.9767 - val_loss: 1.0983 - 2s/epoch - 21ms/step
Epoch 6/100
82/82 - 2s - loss: 0.9767 - val_loss: 1.0982 - 2s/epoch - 19ms/step
Epoch 7/100
82/82 - 2s - loss: 0.9763 - val_loss: 1.0954 - 2s/epoch - 19ms/step
Epoch 8/100
82/82 - 2s - loss: 0.9764 - val_loss: 1.0963 - 2s/epoch - 19ms/step
Epoch 9/100
82/82 - 2s - loss: 0.9764 - val_loss: 1.0963 - 2s/epoch - 19ms/step
Epoch 10/100
82/82 - 2s - loss: 0.9760 - val_loss: 1.0954 - 2s/epoch - 19ms/step
Epoch 11/100
82/82 - 2s - loss: 0.9758 - val_loss: 1.0984 - 2s/epoch - 19ms/step
Epoch 12/100
82/82 - 2s - loss: 0.9760 - val_loss: 1.0954 - 2s/epoch - 20ms/step
Epoch 13/100
82/82 - 2s - loss: 0.975

Epoch 1/100
82/82 - 6s - loss: 0.9774 - val_loss: 1.0961 - 6s/epoch - 78ms/step
Epoch 2/100
82/82 - 2s - loss: 0.9760 - val_loss: 1.0961 - 2s/epoch - 19ms/step
Epoch 3/100
82/82 - 2s - loss: 0.9758 - val_loss: 1.0966 - 2s/epoch - 19ms/step
Epoch 4/100
82/82 - 2s - loss: 0.9752 - val_loss: 1.0991 - 2s/epoch - 19ms/step
Epoch 5/100
82/82 - 2s - loss: 0.9734 - val_loss: 1.1017 - 2s/epoch - 19ms/step
Epoch 6/100
82/82 - 2s - loss: 0.9727 - val_loss: 1.0971 - 2s/epoch - 18ms/step
Epoch 7/100
82/82 - 2s - loss: 0.9700 - val_loss: 1.0996 - 2s/epoch - 19ms/step
Epoch 8/100
82/82 - 2s - loss: 0.9663 - val_loss: 1.0979 - 2s/epoch - 20ms/step
Epoch 9/100
82/82 - 2s - loss: 0.9606 - val_loss: 1.1171 - 2s/epoch - 20ms/step
Epoch 10/100
82/82 - 2s - loss: 0.9544 - val_loss: 1.1071 - 2s/epoch - 20ms/step
Epoch 11/100
82/82 - 2s - loss: 0.9461 - val_loss: 1.1183 - 2s/epoch - 19ms/step
Epoch 12/100
82/82 - 2s - loss: 0.9295 - val_loss: 1.1134 - 2s/epoch - 20ms/step
Epoch 13/100
82/82 - 2s - loss: 0.917

Epoch 1/100
82/82 - 6s - loss: 0.9800 - val_loss: 1.0971 - 6s/epoch - 77ms/step
Epoch 2/100
82/82 - 2s - loss: 0.9763 - val_loss: 1.0973 - 2s/epoch - 19ms/step
Epoch 3/100
82/82 - 2s - loss: 0.9756 - val_loss: 1.0988 - 2s/epoch - 19ms/step
Epoch 4/100
82/82 - 2s - loss: 0.9745 - val_loss: 1.1028 - 2s/epoch - 19ms/step
Epoch 5/100
82/82 - 2s - loss: 0.9732 - val_loss: 1.0996 - 2s/epoch - 19ms/step
Epoch 6/100
82/82 - 2s - loss: 0.9725 - val_loss: 1.1010 - 2s/epoch - 20ms/step
Epoch 7/100
82/82 - 2s - loss: 0.9696 - val_loss: 1.1096 - 2s/epoch - 20ms/step
Epoch 8/100
82/82 - 2s - loss: 0.9712 - val_loss: 1.1092 - 2s/epoch - 19ms/step
Epoch 9/100
82/82 - 2s - loss: 0.9642 - val_loss: 1.1114 - 2s/epoch - 19ms/step
Epoch 10/100
82/82 - 2s - loss: 0.9569 - val_loss: 1.1075 - 2s/epoch - 19ms/step
Epoch 11/100
82/82 - 2s - loss: 0.9484 - val_loss: 1.1089 - 2s/epoch - 19ms/step
Epoch 12/100
82/82 - 2s - loss: 0.9349 - val_loss: 1.1150 - 2s/epoch - 19ms/step
Epoch 13/100
82/82 - 2s - loss: 0.916

Epoch 1/100
82/82 - 6s - loss: 0.9761 - val_loss: 1.0961 - 6s/epoch - 77ms/step
Epoch 2/100
82/82 - 2s - loss: 0.9761 - val_loss: 1.0958 - 2s/epoch - 19ms/step
Epoch 3/100
82/82 - 2s - loss: 0.9760 - val_loss: 1.0960 - 2s/epoch - 20ms/step
Epoch 4/100
82/82 - 2s - loss: 0.9760 - val_loss: 1.0962 - 2s/epoch - 20ms/step
Epoch 5/100
82/82 - 2s - loss: 0.9760 - val_loss: 1.0960 - 2s/epoch - 20ms/step
Epoch 6/100
82/82 - 2s - loss: 0.9761 - val_loss: 1.0960 - 2s/epoch - 21ms/step
Epoch 7/100
82/82 - 2s - loss: 0.9760 - val_loss: 1.0961 - 2s/epoch - 20ms/step
Epoch 8/100
82/82 - 2s - loss: 0.9760 - val_loss: 1.0961 - 2s/epoch - 19ms/step
Epoch 9/100
82/82 - 2s - loss: 0.9760 - val_loss: 1.0961 - 2s/epoch - 20ms/step
Epoch 10/100
82/82 - 2s - loss: 0.9760 - val_loss: 1.0959 - 2s/epoch - 19ms/step
Epoch 11/100
82/82 - 2s - loss: 0.9760 - val_loss: 1.0958 - 2s/epoch - 19ms/step
Epoch 12/100
82/82 - 2s - loss: 0.9760 - val_loss: 1.0958 - 2s/epoch - 19ms/step
Epoch 13/100
82/82 - 2s - loss: 0.976

...

Epoch 1/100
82/82 - 6s - loss: 0.9852 - val_loss: 1.1105 - 6s/epoch - 71ms/step
Epoch 2/100
82/82 - 2s - loss: 0.9804 - val_loss: 1.1180 - 2s/epoch - 19ms/step
Epoch 3/100
82/82 - 2s - loss: 0.9791 - val_loss: 1.1028 - 2s/epoch - 19ms/step
Epoch 4/100
82/82 - 2s - loss: 0.9781 - val_loss: 1.0982 - 2s/epoch - 19ms/step
Epoch 5/100
82/82 - 2s - loss: 0.9774 - val_loss: 1.1041 - 2s/epoch - 19ms/step
Epoch 6/100
82/82 - 2s - loss: 0.9771 - val_loss: 1.0959 - 2s/epoch - 19ms/step
Epoch 7/100
82/82 - 2s - loss: 0.9769 - val_loss: 1.0973 - 2s/epoch - 19ms/step
Epoch 8/100
82/82 - 2s - loss: 0.9766 - val_loss: 1.0967 - 2s/epoch - 20ms/step
Epoch 9/100
82/82 - 2s - loss: 0.9766 - val_loss: 1.0983 - 2s/epoch - 19ms/step
Epoch 10/100
82/82 - 1s - loss: 0.9764 - val_loss: 1.1004 - 1s/epoch - 18ms/step
Epoch 11/100
82/82 - 2s - loss: 0.9765 - val_loss: 1.1030 - 2s/epoch - 18ms/step
Epoch 12/100
82/82 - 2s - loss: 0.9764 - val_loss: 1.0974 - 2s/epoch - 19ms/step
Epoch 13/100
82/82 - 2s - loss: 0.976

Epoch 1/100
82/82 - 6s - loss: 0.9773 - val_loss: 1.0964 - 6s/epoch - 71ms/step
Epoch 2/100
82/82 - 2s - loss: 0.9762 - val_loss: 1.0967 - 2s/epoch - 19ms/step
Epoch 3/100
82/82 - 2s - loss: 0.9756 - val_loss: 1.0977 - 2s/epoch - 19ms/step
Epoch 4/100
82/82 - 2s - loss: 0.9753 - val_loss: 1.0975 - 2s/epoch - 19ms/step
Epoch 5/100
82/82 - 2s - loss: 0.9747 - val_loss: 1.0966 - 2s/epoch - 19ms/step
Epoch 6/100
82/82 - 2s - loss: 0.9745 - val_loss: 1.0984 - 2s/epoch - 20ms/step
Epoch 7/100
82/82 - 2s - loss: 0.9734 - val_loss: 1.0979 - 2s/epoch - 20ms/step
Epoch 8/100
82/82 - 2s - loss: 0.9729 - val_loss: 1.1049 - 2s/epoch - 19ms/step
Epoch 9/100
82/82 - 2s - loss: 0.9731 - val_loss: 1.0985 - 2s/epoch - 19ms/step
Epoch 10/100
82/82 - 2s - loss: 0.9719 - val_loss: 1.1007 - 2s/epoch - 19ms/step
Epoch 11/100
82/82 - 2s - loss: 0.9713 - val_loss: 1.1010 - 2s/epoch - 19ms/step
Epoch 12/100
82/82 - 2s - loss: 0.9711 - val_loss: 1.1004 - 2s/epoch - 19ms/step
Epoch 13/100
82/82 - 2s - loss: 0.969

Epoch 1/100
82/82 - 6s - loss: 0.9802 - val_loss: 1.0979 - 6s/epoch - 72ms/step
Epoch 2/100
82/82 - 2s - loss: 0.9769 - val_loss: 1.0965 - 2s/epoch - 18ms/step
Epoch 3/100
82/82 - 2s - loss: 0.9764 - val_loss: 1.0973 - 2s/epoch - 19ms/step
Epoch 4/100
82/82 - 1s - loss: 0.9754 - val_loss: 1.0962 - 1s/epoch - 18ms/step
Epoch 5/100
82/82 - 2s - loss: 0.9752 - val_loss: 1.0983 - 2s/epoch - 18ms/step
Epoch 6/100
82/82 - 2s - loss: 0.9742 - val_loss: 1.0985 - 2s/epoch - 20ms/step
Epoch 7/100
82/82 - 2s - loss: 0.9729 - val_loss: 1.0975 - 2s/epoch - 20ms/step
Epoch 8/100
82/82 - 2s - loss: 0.9726 - val_loss: 1.0987 - 2s/epoch - 19ms/step
Epoch 9/100
82/82 - 2s - loss: 0.9715 - val_loss: 1.0986 - 2s/epoch - 26ms/step
Epoch 10/100
82/82 - 2s - loss: 0.9702 - val_loss: 1.1025 - 2s/epoch - 19ms/step
Epoch 11/100
82/82 - 2s - loss: 0.9694 - val_loss: 1.1044 - 2s/epoch - 18ms/step
Epoch 12/100
82/82 - 2s - loss: 0.9679 - val_loss: 1.1055 - 2s/epoch - 19ms/step
Epoch 13/100
82/82 - 2s - loss: 0.966

Epoch 1/100
82/82 - 6s - loss: 0.9761 - val_loss: 1.0968 - 6s/epoch - 75ms/step
Epoch 2/100
82/82 - 2s - loss: 0.9761 - val_loss: 1.0960 - 2s/epoch - 19ms/step
Epoch 3/100
82/82 - 2s - loss: 0.9760 - val_loss: 1.0973 - 2s/epoch - 19ms/step
Epoch 4/100
82/82 - 2s - loss: 0.9759 - val_loss: 1.0963 - 2s/epoch - 19ms/step
Epoch 5/100
82/82 - 2s - loss: 0.9761 - val_loss: 1.0966 - 2s/epoch - 20ms/step
Epoch 6/100
82/82 - 2s - loss: 0.9761 - val_loss: 1.0959 - 2s/epoch - 20ms/step
Epoch 7/100
82/82 - 2s - loss: 0.9761 - val_loss: 1.0956 - 2s/epoch - 19ms/step
Epoch 8/100
82/82 - 2s - loss: 0.9761 - val_loss: 1.0960 - 2s/epoch - 20ms/step
Epoch 9/100
82/82 - 2s - loss: 0.9760 - val_loss: 1.0955 - 2s/epoch - 20ms/step
Epoch 10/100
82/82 - 2s - loss: 0.9761 - val_loss: 1.0962 - 2s/epoch - 21ms/step
Epoch 11/100
82/82 - 2s - loss: 0.9761 - val_loss: 1.0963 - 2s/epoch - 20ms/step
Epoch 12/100
82/82 - 2s - loss: 0.9761 - val_loss: 1.0961 - 2s/epoch - 19ms/step
Epoch 13/100
82/82 - 2s - loss: 0.976

Epoch 1/100
82/82 - 6s - loss: 0.9762 - val_loss: 1.0969 - 6s/epoch - 75ms/step
Epoch 2/100
82/82 - 2s - loss: 0.9761 - val_loss: 1.0965 - 2s/epoch - 19ms/step
Epoch 3/100
82/82 - 2s - loss: 0.9760 - val_loss: 1.0963 - 2s/epoch - 21ms/step
Epoch 4/100
82/82 - 2s - loss: 0.9758 - val_loss: 1.0961 - 2s/epoch - 20ms/step
Epoch 5/100
82/82 - 2s - loss: 0.9756 - val_loss: 1.0981 - 2s/epoch - 20ms/step
Epoch 6/100
82/82 - 2s - loss: 0.9755 - val_loss: 1.0966 - 2s/epoch - 20ms/step
Epoch 7/100
82/82 - 2s - loss: 0.9752 - val_loss: 1.0956 - 2s/epoch - 20ms/step
Epoch 8/100
82/82 - 2s - loss: 0.9749 - val_loss: 1.0991 - 2s/epoch - 20ms/step
Epoch 9/100
82/82 - 2s - loss: 0.9744 - val_loss: 1.0966 - 2s/epoch - 20ms/step
Epoch 10/100
82/82 - 2s - loss: 0.9738 - val_loss: 1.0967 - 2s/epoch - 21ms/step
Epoch 11/100
82/82 - 2s - loss: 0.9730 - val_loss: 1.1024 - 2s/epoch - 21ms/step
Epoch 12/100
82/82 - 2s - loss: 0.9720 - val_loss: 1.0976 - 2s/epoch - 20ms/step
Epoch 13/100
82/82 - 2s - loss: 0.970

In [ ]:
print(mse_values)


[1.696575788400437, 1.4836267901570335, 1.7022734607464867, 1.708962638406095, 1.5919721724509737, 1.0961801494468337, 1.726414302468425, 1.5745033534170478, 1.0960931381453745, 2.022044649278073, 1.789469490892809, 1.74372749662868, 1.0961718970534142, 1.574620664128314]
